# Sales Forecast — Colab conversion of predict_ml_sales.py

This Colab notebook converts the `predict_ml_sales.py` analysis into a Colab-friendly workflow that:

- Loads sales data (CSV or SQLite),
- Reproduces the advanced feature engineering,
- Builds a TensorFlow model (sequence model) to forecast next-day sales,
- Exports the model as SavedModel and TFLite (float and quantized),
- Packages feature order and scaler parameters for embedding into the Flutter app.

Usage notes:
- Run cells in order. Use the "Runtime > Change runtime type" menu in Colab and select GPU if available.
- The notebook keeps cells modular so you can re-run preprocessing or model training independently.

---

Outline: Environment setup → Upload/Mount → Inspect Script → Data loading/EDA → Feature engineering → TF dataset → Model → Train → Export → Convert to TFLite → Validate → Package → Dart integration snippet



## 1) Environment setup (Colab + GPU + Packages)

This cell installs required packages and verifies the runtime. If you're on Google Colab, enable a GPU runtime: Runtime → Change runtime type → GPU. The install is quiet and idempotent.

Run the next Python cell to install packages and check available GPUs.

In [ ]:
# Install required packages (run once)
!pip install -q tensorflow pandas numpy matplotlib seaborn scikit-learn tflite-support joblib

# Verify GPU
!nvidia-smi || true

import tensorflow as tf
print('TensorFlow version:', tf.__version__)
print('GPUs:', tf.config.list_physical_devices('GPU'))

# Set seeds for reproducibility
import os, random
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
import numpy as np
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)

print('Seeds set.')


## 2) Upload or mount project and inspect `predict_ml_sales.py`

Options:
- Upload files directly via Colab UI (files.upload),
- Mount Google Drive and copy files from your Drive, or
- If running locally, place `predict_ml_sales.py` in the notebook folder and inspect.

Use the next cell to mount Google Drive or use the file upload widget.

In [ ]:
# Option A: Upload file via Colab UI
from google.colab import files
print('If you have predict_ml_sales.py locally, upload it now (optional).')
# uploaded = files.upload()  # Uncomment to use upload widget

# Option B: Mount Google Drive (recommended for larger files)
from google.colab import drive
print('Mounting Drive to /content/drive ...')
drive.mount('/content/drive', force_remount=False)

# Example path if you placed the repo in Drive
# repo_path = '/content/drive/MyDrive/MamaAbbysMNVGMNT'
# !ls -la "{repo_path}" 

# If predict_ml_sales.py was uploaded or copied, show first 200 lines
import os
if os.path.exists('predict_ml_sales.py'):
    print('\nFound predict_ml_sales.py in notebook directory. Showing first 200 lines:')
    !sed -n '1,200p' predict_ml_sales.py
else:
    print('\npredict_ml_sales.py not found in the notebook directory. If you mounted Drive, copy it to /content or change the path.')


## 3) Inspect script structure & refactor plan

We'll reuse the advanced feature engineering from `predict_ml_sales.py` but adapt the training to TensorFlow. The plan:

- Load data into `df` (daily sales totals).
- Reproduce engineered features found in predict_ml_sales.py (lags, rolling stats, cyclical features).
- Create sequences of length `window_size` (e.g., 30) to predict the next day's sales.
- Train a compact Keras model and export it for TFLite conversion.

Run the next cell to import common libraries used below.

In [ ]:
# Common imports for the rest of the notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime

sns.set_style('darkgrid')

print('Libraries imported')


## 4) Data loading and quick EDA

This cell provides three options to load data into `df`:
- Option 1: Load CSV you uploaded (uncomment to use),
- Option 2: Load CSV from Drive path,
- Option 3: Load from a SQLite DB (if you copied the app.db to Colab/Drive) and the `store_sales` table exists.

The expected DataFrame should have at least: `sale_date` (date), `sales` (numeric), `holiday_flag` (0/1), `day_of_week`, and `month` (optional). We'll try CSV first.

In [ ]:
# Option 1: CSV in notebook directory (uncomment when file uploaded)
# df = pd.read_csv('sales_daily.csv', parse_dates=['sale_date'])

# Option 2: CSV in Drive (change path as needed)
drive_csv_path = '/content/drive/MyDrive/MamaAbbysMNVGMNT/sales_daily.csv'
if os.path.exists(drive_csv_path):
    print('Loading CSV from Drive:', drive_csv_path)
    df = pd.read_csv(drive_csv_path, parse_dates=['sale_date'])
else:
    # Option 3: try reading from copied SQLite database if available
    db_path_candidates = [
        '/content/drive/MyDrive/MamaAbbysMNVGMNT/app.db',
        '/content/app.db',
        '/content/drive/MyDrive/app.db'
    ]
    db_found = None
    for p in db_path_candidates:
        if os.path.exists(p):
            db_found = p
            break

    if db_found:
        print('Loading from SQLite DB:', db_found)
        import sqlite3
        with sqlite3.connect(db_found) as conn:
            query = 'SELECT id, sale_date, day_of_week, month, holiday_flag, sales FROM store_sales ORDER BY sale_date'
            df = pd.read_sql_query(query, conn, parse_dates=['sale_date'])
    else:
        raise FileNotFoundError('No CSV or DB found. Upload sales CSV as sales_daily.csv or place app.db in Drive.')

print('Loaded dataframe shape:', df.shape)
print(df.head())
print('\nData types:\n', df.dtypes)

# Quick time range
print('\nDate range:', df['sale_date'].min(), '->', df['sale_date'].max())

# Basic plot of sales over time
plt.figure(figsize=(14, 4))
plt.plot(df['sale_date'], df['sales'], label='sales')
plt.title('Sales over time')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.show()


## 5) Feature engineering (adapted from predict_ml_sales.py)

This cell creates date-based features, cyclical encodings, lag features, rolling statistics, EMAs, ratios, and drops NaNs — matching the heavy feature engineering in the original file but kept deterministic for TensorFlow pipelines.

You can adjust lag windows and rolling windows as needed.

In [ ]:
# Ensure required columns exist
if 'sale_date' not in df.columns or 'sales' not in df.columns:
    raise ValueError('Dataframe must include sale_date and sales columns')

# Copy df to avoid modifying original
df_proc = df.copy()

# Ensure datetime
df_proc['sale_date'] = pd.to_datetime(df_proc['sale_date'])

# Fill missing months or holiday flags if absent
if 'month' not in df_proc.columns:
    df_proc['month'] = df_proc['sale_date'].dt.month
if 'holiday_flag' not in df_proc.columns:
    df_proc['holiday_flag'] = 0
if 'day_of_week' not in df_proc.columns:
    df_proc['day_of_week'] = df_proc['sale_date'].dt.day_name()

# Sort and set index
df_proc = df_proc.sort_values('sale_date').reset_index(drop=True)

# Extract date parts
df_proc['year'] = df_proc['sale_date'].dt.year
df_proc['day'] = df_proc['sale_date'].dt.day
df_proc['quarter'] = df_proc['sale_date'].dt.quarter
df_proc['weekday'] = df_proc['sale_date'].dt.weekday
df_proc['week_of_year'] = df_proc['sale_date'].dt.isocalendar().week
df_proc['day_of_year'] = df_proc['sale_date'].dt.dayofyear

df_proc['is_weekend'] = (df_proc['weekday'] >= 5).astype(int)
df_proc['is_month_start'] = df_proc['sale_date'].dt.is_month_start.astype(int)
df_proc['is_month_end'] = df_proc['sale_date'].dt.is_month_end.astype(int)
df_proc['is_quarter_start'] = df_proc['sale_date'].dt.is_quarter_start.astype(int)
df_proc['is_quarter_end'] = df_proc['sale_date'].dt.is_quarter_end.astype(int)

# Map day_of_week to numeric
day_mapping = {'Monday':0,'Tuesday':1,'Wednesday':2,'Thursday':3,'Friday':4,'Saturday':5,'Sunday':6}
if df_proc['day_of_week'].dtype == object:
    df_proc['day_of_week_encoded'] = df_proc['day_of_week'].map(day_mapping).fillna(df_proc['weekday']).astype(int)
else:
    df_proc['day_of_week_encoded'] = pd.to_numeric(df_proc['day_of_week'], errors='coerce').fillna(df_proc['weekday']).astype(int)

# Cyclical features
df_proc['month_sin'] = np.sin(2*np.pi*df_proc['month']/12)
df_proc['month_cos'] = np.cos(2*np.pi*df_proc['month']/12)
df_proc['day_sin'] = np.sin(2*np.pi*df_proc['day']/31)
df_proc['day_cos'] = np.cos(2*np.pi*df_proc['day']/31)
df_proc['weekday_sin'] = np.sin(2*np.pi*df_proc['weekday']/7)
df_proc['weekday_cos'] = np.cos(2*np.pi*df_proc['weekday']/7)
df_proc['quarter_sin'] = np.sin(2*np.pi*df_proc['quarter']/4)
df_proc['quarter_cos'] = np.cos(2*np.pi*df_proc['quarter']/4)
df_proc['week_sin'] = np.sin(2*np.pi*df_proc['week_of_year']/52)
df_proc['week_cos'] = np.cos(2*np.pi*df_proc['week_of_year']/52)

# Lag features
for lag in [1,2,3,7,14,21,30]:
    df_proc[f'sales_lag{lag}'] = df_proc['sales'].shift(lag)

# Rolling stats
for window in [3,7,14,21,30]:
    df_proc[f'sales_ma{window}'] = df_proc['sales'].rolling(window=window).mean()
    df_proc[f'sales_std{window}'] = df_proc['sales'].rolling(window=window).std()
    df_proc[f'sales_min{window}'] = df_proc['sales'].rolling(window=window).min()
    df_proc[f'sales_max{window}'] = df_proc['sales'].rolling(window=window).max()
    df_proc[f'sales_median{window}'] = df_proc['sales'].rolling(window=window).median()

# EMA
for span in [7,14,30]:
    df_proc[f'sales_ema{span}'] = df_proc['sales'].ewm(span=span).mean()

# Trend
df_proc['sales_diff1'] = df_proc['sales'].diff(1)
df_proc['sales_diff7'] = df_proc['sales'].diff(7)
df_proc['sales_pct_change1'] = df_proc['sales'].pct_change(1)
df_proc['sales_pct_change7'] = df_proc['sales'].pct_change(7)

# Interactions and polynomial
df_proc['month_holiday'] = df_proc['month'] * df_proc['holiday_flag']
df_proc['weekday_holiday'] = df_proc['weekday'] * df_proc['holiday_flag']
df_proc['is_weekend_holiday'] = df_proc['is_weekend'] * df_proc['holiday_flag']
df_proc['month_squared'] = df_proc['month']**2
df_proc['weekday_squared'] = df_proc['weekday']**2
df_proc['day_squared'] = df_proc['day']**2

# Ratios and volatility
df_proc['sales_to_ma7_ratio'] = df_proc['sales'] / (df_proc['sales_ma7'] + 1e-8)
df_proc['sales_to_ma30_ratio'] = df_proc['sales'] / (df_proc['sales_ma30'] + 1e-8)
df_proc['ma7_to_ma30_ratio'] = df_proc['sales_ma7'] / (df_proc['sales_ma30'] + 1e-8)

df_proc['sales_volatility_7'] = df_proc['sales'].rolling(window=7).std() / (df_proc['sales'].rolling(window=7).mean() + 1e-8)
df_proc['sales_volatility_30'] = df_proc['sales'].rolling(window=30).std() / (df_proc['sales'].rolling(window=30).mean() + 1e-8)

# Detrended
if 'sales_ma30' in df_proc.columns:
    df_proc['sales_detrended'] = df_proc['sales'] - df_proc['sales_ma30']

# Drop rows with NaNs introduced by lags/rolling
df_proc = df_proc.dropna().reset_index(drop=True)

print('Processed shape:', df_proc.shape)
print('Columns:', df_proc.columns.tolist()[:40])

# Keep a ordered feature list (exclude metadata and target)
exclude_cols = ['id','sale_date','day_of_week','sales']
feature_cols = [c for c in df_proc.columns if c not in exclude_cols]
print('Feature count:', len(feature_cols))

# Show the first rows
df_proc.head()


## 6) Create windowed sequences (tf.data) and scale features

We'll create sequences of length `window_size` (e.g., 30) using the engineered features to predict the next-day sales. We'll scale features using scikit-learn's StandardScaler and save scaler params for use in Flutter (as JSON/npz).

In [ ]:
from sklearn.preprocessing import StandardScaler
import joblib

# Parameters
window_size = 30  # days to use as input
forecast_horizon = 1  # predict next day

# Prepare X (3D) and y (1D)
X_raw = df_proc[feature_cols].values
y_raw = df_proc['sales'].values

# Create scaler and fit on train portion (time-based split)
split_idx = int(len(X_raw) * 0.8)
scaler = StandardScaler()
scaler.fit(X_raw[:split_idx])
X_scaled = scaler.transform(X_raw)

# Save scaler params
os.makedirs('export', exist_ok=True)
joblib.dump(scaler, 'export/scaler.joblib')
np.savez('export/scaler_params.npz', mean=scaler.mean_, scale=scaler.scale_)

# Build sequences
def build_sequences(X, y, window_size):
    Xs, ys = [], []
    for i in range(window_size, len(X)):
        Xs.append(X[i-window_size:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = build_sequences(X_scaled, y_raw, window_size)
print('X_seq shape:', X_seq.shape)
print('y_seq shape:', y_seq.shape)

# Train/val/test split (time-ordered)
train_size = int(0.8 * len(X_seq))
X_train, y_train = X_seq[:train_size], y_seq[:train_size]
X_val, y_val = X_seq[train_size:], y_seq[train_size:]

print('Train shape:', X_train.shape, y_train.shape)
print('Val shape:', X_val.shape, y_val.shape)

# Create tf.data datasets
import tensorflow as tf
batch_size = 32
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024, seed=seed).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

print('Datasets ready')


## 7) Model architecture (Keras)

We'll build a compact model suitable for TFLite: a small Conv1D + Dense or LSTM. Conv1D is typically smaller and faster on-device. The model inputs shape: (window_size, n_features).

In [ ]:
def build_model(input_shape):
    from tensorflow.keras import layers, models
    inp = layers.Input(shape=input_shape)
    x = layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inp)
    x = layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    out = layers.Dense(1, activation='linear')(x)
    model = models.Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

input_shape = (window_size, len(feature_cols))
model = build_model(input_shape)
model.summary()


# Small utility to compute RMSE and R2
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

def r2_score_np(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    return 1 - ss_res/ss_tot



## 8) Train model with callbacks and visualize learning

We'll use EarlyStopping and ModelCheckpoint to save the best model. Training is intentionally small (few epochs) to keep Colab runs quick; increase epochs for better performance.

In [ ]:
# Callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
os.makedirs('export', exist_ok=True)
ckpt_path = 'export/best_model.h5'
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    ModelCheckpoint(ckpt_path, monitor='val_loss', save_best_only=True)
]

# Train
epochs = 50
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks
)

# Plot training curves
plt.figure(figsize=(10,4))
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.title('Training curves')
plt.show()

# Load best model saved by checkpoint
model.load_weights(ckpt_path)

# Evaluate on validation
val_preds = model.predict(X_val).flatten()
val_mse = np.mean((val_preds - y_val)**2)
val_rmse = np.sqrt(val_mse)
val_mae = np.mean(np.abs(val_preds - y_val))
val_r2 = r2_score_np(y_val, val_preds)
print(f'Validation RMSE: {val_rmse:.4f}, MAE: {val_mae:.4f}, R2: {val_r2:.4f}')

# Plot predicted vs actual (last 200 points)
plt.figure(figsize=(14,5))
plt.plot(y_val[-200:], label='actual')
plt.plot(val_preds[-200:], label='predicted')
plt.legend()
plt.title('Actual vs Predicted (validation tail)')
plt.show()


## 9) Export SavedModel and HDF5

We'll save the Keras model in SavedModel format and HDF5 (for convenience). We'll also save the feature order and scaler params to `export/` so the Flutter app knows the input order/normalization.

In [ ]:
# Export SavedModel
saved_model_dir = 'export/saved_model'
model.save(saved_model_dir, include_optimizer=False)
print('SavedModel saved to', saved_model_dir)

# Save HDF5
model.save('export/model.h5')
print('HDF5 saved to export/model.h5')

# Save feature order
import json
with open('export/feature_order.json', 'w') as f:
    json.dump(feature_cols, f)
print('Saved feature_order.json')

# Ensure scaler_params.npz exists (saved earlier)
print('Export folder contents:')
!ls -la export


## 10) Convert SavedModel to TFLite (float & quantized)

We'll convert the saved model to TFLite float model and then use post-training quantization. For full integer quantization, we provide a representative dataset generator built from training sequences.

In [ ]:
# Convert to TFLite (float)
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
tflite_model = converter.convert()
open('export/model.tflite', 'wb').write(tflite_model)
print('Float TFLite saved to export/model.tflite')

# Representative dataset for quantization (use a subset of training data)
def representative_dataset_gen():
    for i in range(min(1000, X_train.shape[0])):
        sample = X_train[i:i+1].astype(np.float32)
        yield [sample]

# Post-training quantization (dynamic range)
converter_quant = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter_quant.optimizations = [tf.lite.Optimize.DEFAULT]
converter_quant.representative_dataset = representative_dataset_gen
converter_quant.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set input and output types to int8 for full integer quantization
converter_quant.inference_input_type = tf.int8
converter_quant.inference_output_type = tf.int8
try:
    tflite_quant = converter_quant.convert()
    open('export/model_quant.tflite', 'wb').write(tflite_quant)
    print('Quantized TFLite saved to export/model_quant.tflite')
except Exception as e:
    print('Quantization failed (fall back to dynamic range quantization):', e)
    # Try dynamic range quantization
    converter_dr = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
    converter_dr.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_dr = converter_dr.convert()
    open('export/model_quant_dynamic.tflite', 'wb').write(tflite_dr)
    print('Dynamic-range quantized TFLite saved to export/model_quant_dynamic.tflite')

print('\nExported files:')
!ls -la export


## 11) Validate TFLite model locally in Python

We'll run a few inference checks with both the Keras model and the TFLite model on the same validation samples and compare outputs.

In [ ]:
# Helper to run tflite model
import tensorflow as tf

def run_tflite_model(tflite_path, X_input):
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    # Accept either int8 or float32 input types
    inp_dtype = input_details[0]['dtype']
    X = X_input.astype(np.float32)
    if inp_dtype == np.int8:
        # map float into int8 range using 127*X (not ideal, representative dataset needed for proper scaling)
        X = (X * 1.0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], X)
    interpreter.invoke()
    out = interpreter.get_tensor(output_details[0]['index'])
    return out

# Pick small slice from validation
sample_X = X_val[:20].astype(np.float32)
keras_out = model.predict(sample_X).flatten()

# Float TFLite
tflite_float_path = 'export/model.tflite'
if os.path.exists(tflite_float_path):
    tflite_out = run_tflite_model(tflite_float_path, sample_X)
    print('Keras vs TFLite float first 5 preds:')
    for k, t in zip(keras_out[:5], tflite_out.flatten()[:5]):
        print(f'{k:.4f}  |  {t:.4f}')
else:
    print('Float TFLite not found')

# Quantized
tflite_q_path = 'export/model_quant.tflite'
if not os.path.exists(tflite_q_path):
    tflite_q_path = 'export/model_quant_dynamic.tflite' if os.path.exists('export/model_quant_dynamic.tflite') else None

if tflite_q_path and os.path.exists(tflite_q_path):
    try:
        tflite_q_out = run_tflite_model(tflite_q_path, sample_X)
        print('\nKeras vs TFLite quant first 5 preds:')
        for k, t in zip(keras_out[:5], tflite_q_out.flatten()[:5]):
            print(f'{k:.4f}  |  {t:.4f}')
    except Exception as e:
        print('Quant TFLite run failed:', e)
else:
    print('Quantized TFLite not found')


## 12) Package artifacts for Flutter

We'll create a zip containing:
- model.tflite (or quantized),
- feature_order.json,
- scaler_params.npz,
- a short `metadata.json` describing input shape and feature order.

Download the zip or copy it to Drive for integration into your Flutter app under `models/`.

In [ ]:
import json, zipfile

metadata = {
    'input_shape': [window_size, len(feature_cols)],
    'feature_order': feature_cols,
    'scaler': 'StandardScaler',
    'dtype': 'float32'
}
with open('export/metadata.json', 'w') as f:
    json.dump(metadata, f)

zip_path = 'export/artifacts.zip'
with zipfile.ZipFile(zip_path, 'w') as z:
    for fname in ['feature_order.json', 'scaler_params.npz', 'metadata.json']:
        if os.path.exists(f'export/{fname}'):
            z.write(f'export/{fname}', arcname=fname)
    # prefer quantized if available
    if os.path.exists('export/model_quant.tflite'):
        z.write('export/model_quant.tflite', arcname='model.tflite')
    elif os.path.exists('export/model_quant_dynamic.tflite'):
        z.write('export/model_quant_dynamic.tflite', arcname='model.tflite')
    elif os.path.exists('export/model.tflite'):
        z.write('export/model.tflite', arcname='model.tflite')

print('Packaged artifacts to', zip_path)

# Optionally copy to Drive
drive_export_dir = '/content/drive/MyDrive/MamaAbbysMNVGMNT/export'
try:
    os.makedirs(drive_export_dir, exist_ok=True)
    import shutil
    shutil.copy(zip_path, os.path.join(drive_export_dir, 'artifacts.zip'))
    print('Copied artifacts.zip to Drive:', drive_export_dir)
except Exception as e:
    print('Could not copy to Drive:', e)

# Provide download link in Colab
from google.colab import files
print('Starting download...')
files.download(zip_path)


## 13) Dart integration snippet (for `sales_page.dart` Forecast tab)

Paste this into your Flutter project (e.g., a `ForecastService`) that loads `assets/model.tflite` and runs inference using `tflite_flutter` and `tflite_flutter_helper`.

Note: you'll need to include the TFLite file in `pubspec.yaml` under `assets:` and add `tflite_flutter` and `tflite_flutter_helper` packages to `pubspec.yaml`.

Example (pseudo-code):

```dart
// In ForecastService
import 'dart:typed_data';
import 'package:tflite_flutter/tflite_flutter.dart';

class ForecastService {
  late Interpreter _interpreter;
  List<String> featureOrder; // load from feature_order.json
  List<double> scalerMean, scalerScale; // load from npz converted to json

  Future<void> loadModel() async {
    _interpreter = await Interpreter.fromAsset('model.tflite');
  }

  Future<double> predictNext(List<double> featuresWindow) async {
    // featuresWindow: flattened window_size * feature_count in row-major
    final input = Float32List.fromList(featuresWindow); // shape [1, window, features]
    final output = Float32List(1);
    _interpreter.run(input, output);
    return output[0];
  }
}
```

In `sales_page.dart`, call your ForecastService's `forecastNext30Days()` (similar to existing import) and return a list of `DailyForecast(date, predictedSales)` for display in the Forecast tab.



## 14) Optional: Serve model via a small REST API (Flask/FastAPI)

If you prefer remote inference instead of embedding the model, you can run a small Flask or FastAPI server in Colab (not recommended for production). This section provides a sample FastAPI snippet (commented) and notes about deploying to a proper server or Cloud Run.

(Implementation left as an exercise — uncomment to use in short-lived demos.)

## 15) Reproducibility & environment capture

Save a `requirements.txt` snapshot for reproducibility (note Colab uses many preinstalled packages). Also include seed reminders and runtime notes.

In [ ]:
# Save pip freeze
!pip freeze > export/requirements.txt
print('Saved pip freeze to export/requirements.txt')

# Final note to user
print('\nNotebook ready. Run cells in order. After export, copy model artifacts into your Flutter project `models/` and update pubspec.yaml with the model asset.')
